# **Valkey Search - Vector Search**

This notebook walks through an example of using Vector Search in OCI Cache (Redis/Valkey) running on Oracle cloud Infrastructure.

- Notebook is deployed in the same private subnet as the OCI Cache cluster
- Notebook has NAT gatway attached in the private subnet
- Notebook has a conda env installed, with the below additional packages installed (langchain, valkey, etc)
- Provision a OCI Cache cluster 8.1. Fetch the primary endpoint

## **Installs / Imports**

In [ ]:
#! pip install langchain~=0.3 langchain_community~=0.3 langchain-aws~=0.2 valkey~=6.1 boto3~=1.40 beautifulsoup4~=4.12

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import valkey
from valkey.commands.search.query import Query
from valkey.commands.search.field import VectorField
import numpy as np
import oci

## **Scrape a FAQ as example**

In [ ]:
oci_cache_faq_url = "https://www.oracle.com/cloud/cache/faq/"

loader = WebBaseLoader(web_path=oci_cache_faq_url)
pages = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", " "],
    chunk_size=1000,
    chunk_overlap=150)
chunks = text_splitter.split_documents(pages)

## Print example chunks
print(chunks[2])
print("---")
print(chunks[3])

## **Establish connection to OCI Cache - Valkey**

Provision an OCI Cache Cluster with Version 8.1 (currently in LA)

In [ ]:
valkey_cluster_primary_endpoint = "Add your primary endpoint here"

valkey_client = valkey.Valkey(
    host=valkey_cluster_primary_endpoint,
    port=6379,
    ssl=True,
    ssl_cert_reqs="none",
)

# The ping should return "True"
valkey_client.ping()

## **Test GenAI Embedding model**

The below code only test the connection from this notebook to the GenAI Embedding model hosted on Oracle Cloud. Auth is based on the config file, which you can extract from your OCI Console

In [ ]:
#set two parameters:
endpoint = "https://inference.generativeai.eu-frankfurt-1.oci.oraclecloud.com"
compartment_ocid = "add your compartment OCID here"


#auth
config = oci.config.from_file('~/config', "DEFAULT")

endpoint = "https://inference.generativeai.eu-frankfurt-1.oci.oraclecloud.com"
generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(config=config, service_endpoint=endpoint, retry_strategy=oci.retry.NoneRetryStrategy(), timeout=(10,240))
inputs = ["some text"]
embed_text_detail = oci.generative_ai_inference.models.EmbedTextDetails()
embed_text_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(model_id="cohere.embed-english-v3.0")
embed_text_detail.inputs = inputs
embed_text_detail.truncate = "NONE"
embed_text_detail.compartment_id = compartment_ocid
embed_text_response = generative_ai_inference_client.embed_text(embed_text_detail)
print("**************************Embed Texts Result**************************")
print(embed_text_response.data)



## **Iterate over text in chuncks, create embeddings and load to Valkey**

The below code will embed the GenAI embedding as small function (with text/chunks as input and the embeddings as output). Then, the code will interate over the chunks, create the embeddings, and use the Valkey clien to add the embeddings to the Valkey cluster.

In [ ]:
def create_embeddings(inputs):

    #set two variables:
    endpoint = "https://inference.generativeai.eu-frankfurt-1.oci.oraclecloud.com"
    compartment_ocid = "add your compartement ocid here"
    
    config = oci.config.from_file('~/config', "DEFAULT")
    endpoint = endpoint
    generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(config=config, service_endpoint=endpoint, retry_strategy=oci.retry.NoneRetryStrategy(), timeout=(10,240))
    embed_text_detail = oci.generative_ai_inference.models.EmbedTextDetails()
    embed_text_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(model_id="cohere.embed-english-v3.0")
    embed_text_detail.inputs = inputs
    embed_text_detail.truncate = "NONE"
    embed_text_detail.compartment_id = compartment_ocid
    embed_text_response = generative_ai_inference_client.embed_text(embed_text_detail)
    
    return embed_text_response.data.embeddings

#embeddings to bytes function
to_bytes = lambda x: np.array(x, dtype=np.float32).tobytes()

# interate over the chunks, create the embeddings, and "hset" to Valkey cluster
for i, chunk in enumerate(chunks):
    content = chunk.page_content
    vector_embedding = to_bytes(create_embeddings([content]))
    valkey_client.hset(f'chunkId:{i}', mapping={'embed': vector_embedding, 'text': content})

## **Create a vector index**

In [ ]:
valkey_client.ft(index_name="embedding_index_v1").create_index([
    VectorField(
        "embed",
        "HNSW",
        {
            "TYPE": "FLOAT32",
            "DIM": 1024,
            "DISTANCE_METRIC": "COSINE",
        })
    ])

#This should return "OK"

## **Run an example query**

In [ ]:
def query_valkey(question):

    #question to embedding to byte
    query_vec = to_bytes(create_embeddings([question]))

    # the query for Valkey. Only returns 1 text. Change to 2 or higher.
    query = (
        Query("*=>[KNN 1 @embed $vec as score]")
         .return_fields("id", "score")
         .dialect(2)
    )
    
    # Find K similar document chunks
    query_params = {"vec": query_vec}

    #search valkey cluster
    results = valkey_client.ft(index_name="embedding_index_v1").search(query, query_params).docs

    for idx, item in enumerate(results):
        results_in_text = (valkey_client.hget(item.id, 'text'))
    
    return results_in_text
    

## **Return top 1 result as example using Vector Search in OCI Cache (Valkey 8.1)**

In [ ]:
question1 = "What version does OCI Cache support?"
question2 = "Why would you use OCI Cache?"

results_in_text = query_valkey(question1)
results_in_text

